## Data Ingestion

In [1]:
import pandas as pd

column_names = ['id', 'text', 'label']
weibo_train = pd.read_csv("/kaggle/input/weibo-dataset/weibo.train", sep="\t", 
                               header=None, names=column_names)
weibo_test = pd.read_csv("/kaggle/input/weibo-dataset/weibo.test", sep="\t", 
                              header=None, names=column_names)
weibo_dev = pd.read_csv("/kaggle/input/weibo-dataset/weibo.dev", sep="\t", 
                             header=None, names=column_names)

In [2]:
weibo_test.dropna(inplace=True)

In [3]:
weibo_train.head()

,id,text,label
0,3910552407639880,【教育部新规拟要求高校辅导员必须是中共党员】教育部日前在官网公布了《普通高等学校辅导员队伍建...,non-rumor
1,3536960792696376,【民生】特大好消息：继北京出台95岁老人看病不要钱后，山东泰安宣布100岁可免费登泰山，初步...,false
2,3921542276460163,【被疯传的投影舞 看哭了[泪]】这段不足4分钟的舞蹈由8位舞者共同完成，舞出一对情侣从相识到...,non-rumor
3,3922114622540868,#突发#【深圳一处工业园区被曝遭山体滑坡掩埋】据网友曝料，深圳光明新区红坳村柳溪工业园发生山...,non-rumor
4,3911293176451011,【“朝阳群众”注册人数已达13万！[赞]】他们大隐隐于市，不露声色却屡建奇功……目前登记在册...,non-rumor


In [4]:
weibo_train.shape, weibo_test.shape, weibo_dev.shape

((3147, 3), (1049, 3), (467, 3))

## Data Preparation

In [5]:
LABELS = weibo_train['label'].unique().tolist()
train_label = weibo_train['label'].replace(LABELS, [0, 1]).tolist()

/tmp/ipykernel_36/2429078884.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_label = weibo_train['label'].replace(LABELS, [0, 1]).tolist()


In [6]:
LABELS

['non-rumor', 'false']

In [7]:
train_data = weibo_train['text'].tolist()

In [8]:
dev_data = weibo_dev['text'].tolist()
dev_label = weibo_dev['label'].replace(LABELS, [0, 1]).tolist()

/tmp/ipykernel_36/358168363.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dev_label = weibo_dev['label'].replace(LABELS, [0, 1]).tolist()


In [9]:
test_data = weibo_test['text'].tolist()
test_label = weibo_test['label'].replace(LABELS, [0, 1]).tolist()

/tmp/ipykernel_36/6925416.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_label = weibo_test['label'].replace(LABELS, [0, 1]).tolist()


In [10]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

class ClassificationDataset(Dataset):
    def __init__(self, texts: list[str], labels: list[int], tokenizer, max_length: int):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            text, 
            add_special_tokens=True, 
            max_length=self.max_length, 
            return_token_type_ids=False, 
            padding="max_length",
            truncation=True, 
            return_attention_mask=True, 
            return_tensors="pt"
        )
        
        return {
            "input_ids": encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [11]:
from transformers import RobertaTokenizer, RobertaModel, AdamW

max_length = 128

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
train_dataset = ClassificationDataset(train_data, train_label, tokenizer, max_length)
dev_dataset = ClassificationDataset(dev_data, dev_label, tokenizer, max_length)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)

In [13]:
test_dataset = ClassificationDataset(test_data, test_label, tokenizer, max_length)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

## Model Preparation

In [14]:
num_classes = 2

In [15]:
class RobertaClassifier(nn.Module):
    def __init__(self, roberta_model, num_classes, dropout=0.1):
        super(RobertaClassifier, self).__init__()
        self.roberta = roberta_model
        self.dropout = nn.Dropout(dropout)
        self.ffn = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.ffn(x)
        return logits

In [16]:
roberta_model = RobertaModel.from_pretrained('roberta-base')

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
model = RobertaClassifier(roberta_model, num_classes)

In [18]:
learning_rate = 2e-5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

RobertaClassifier(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
from tqdm import tqdm

def calculate_metrics(y_true, y_pred, y_pred_proba, num_classes):
    if num_classes == 2:
        metrics = {
            'accuracy': accuracy_score(y_true, y_pred),
            'precision': precision_score(y_true, y_pred),
            'recall': recall_score(y_true, y_pred),
            'f1': f1_score(y_true, y_pred)
        }
    else:
        metrics = {
            'accuracy': accuracy_score(y_true, y_pred),
            'precision': precision_score(y_true, y_pred, average='weighted'),
            'recall': recall_score(y_true, y_pred, average='weighted'),
            'f1': f1_score(y_true, y_pred, average='weighted')
        }
    
    # Calculate ROC-AUC score
    if num_classes == 2:
        metrics['roc_auc'] = roc_auc_score(y_true, y_pred_proba[:, 1])
    else:
        try:
            metrics['roc_auc'] = roc_auc_score(y_true, y_pred_proba, multi_class='ovr', average='weighted')
        except:
            metrics['roc_auc'] = None
            
    return metrics

In [20]:
def evaluate(model, data_loader, device, num_classes):
    model.eval()
    all_labels = []
    all_predictions = []
    all_predictions_proba = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask)
            probabilities = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
            all_predictions_proba.extend(probabilities.cpu().numpy())

    all_labels = np.array(all_labels)
    all_predictions = np.array(all_predictions)
    all_predictions_proba = np.array(all_predictions_proba)

    return calculate_metrics(all_labels, all_predictions, all_predictions_proba, num_classes)

In [21]:
def train(model, train_loader, val_loader, criterion, optimizer, device, num_epochs):
    best_val_metrics = {'f1': 0.0}
    history = {'train': [], 'val': []}

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        all_train_labels = []
        all_train_predictions = []
        all_train_predictions_proba = []

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            probabilities = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            all_train_labels.extend(labels.cpu().numpy())
            all_train_predictions.extend(predicted.cpu().numpy())
            all_train_predictions_proba.extend(probabilities.cpu().detach().numpy())

        # Calculate training metrics
        all_train_labels = np.array(all_train_labels)
        all_train_predictions = np.array(all_train_predictions)
        all_train_predictions_proba = np.array(all_train_predictions_proba)
        train_metrics = calculate_metrics(all_train_labels, all_train_predictions, 
                                       all_train_predictions_proba, num_classes)
        
        # Validation phase
        val_metrics = evaluate(model, val_loader, device, num_classes)
        
        # Store metrics history
        history['train'].append({
            'loss': train_loss / len(train_loader),
            **train_metrics
        })
        history['val'].append(val_metrics)

        # Print epoch results
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("Training Metrics:")
        print(f"Loss: {train_loss/len(train_loader):.4f}")
        for metric, value in train_metrics.items():
            print(f"{metric.capitalize()}: {value:.4f}")
        
        print("\nValidation Metrics:")
        for metric, value in val_metrics.items():
            print(f"{metric.capitalize()}: {value:.4f}")
    
    return history

In [22]:
num_epochs = 5

optimizer = AdamW(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [23]:
history = train(model, train_loader, dev_loader, criterion, optimizer, device, num_epochs)

Epoch 1/5: 100%|██████████| 99/99 [00:42<00:00,  2.35it/s]



Epoch 1/5
Training Metrics:
Loss: 0.6725
Accuracy: 0.5688
Precision: 0.5785
Recall: 0.4795
F1: 0.5244
Roc_auc: 0.6150

Validation Metrics:
Accuracy: 0.6510
Precision: 0.6030
Recall: 0.8707
F1: 0.7125
Roc_auc: 0.7431


Epoch 2/5: 100%|██████████| 99/99 [00:37<00:00,  2.63it/s]



Epoch 2/5
Training Metrics:
Loss: 0.6138
Accuracy: 0.6670
Precision: 0.6446
Recall: 0.7314
F1: 0.6853
Roc_auc: 0.7213

Validation Metrics:
Accuracy: 0.6916
Precision: 0.7075
Recall: 0.6466
F1: 0.6757
Roc_auc: 0.7787


Epoch 3/5: 100%|██████████| 99/99 [00:37<00:00,  2.63it/s]



Epoch 3/5
Training Metrics:
Loss: 0.5759
Accuracy: 0.7029
Precision: 0.6895
Recall: 0.7288
F1: 0.7086
Roc_auc: 0.7686

Validation Metrics:
Accuracy: 0.7323
Precision: 0.6989
Recall: 0.8103
F1: 0.7505
Roc_auc: 0.8201


Epoch 4/5: 100%|██████████| 99/99 [00:37<00:00,  2.63it/s]



Epoch 4/5
Training Metrics:
Loss: 0.5413
Accuracy: 0.7328
Precision: 0.7139
Recall: 0.7692
F1: 0.7405
Roc_auc: 0.8006

Validation Metrics:
Accuracy: 0.7238
Precision: 0.7373
Recall: 0.6897
F1: 0.7127
Roc_auc: 0.8158


Epoch 5/5: 100%|██████████| 99/99 [00:37<00:00,  2.63it/s]



Epoch 5/5
Training Metrics:
Loss: 0.4921
Accuracy: 0.7687
Precision: 0.7506
Recall: 0.7987
F1: 0.7739
Roc_auc: 0.8406

Validation Metrics:
Accuracy: 0.7495
Precision: 0.6936
Recall: 0.8879
F1: 0.7788
Roc_auc: 0.8488


In [24]:
import pandas as pd

pd.DataFrame(history['train'])

,loss,accuracy,precision,recall,f1,roc_auc
0,0.672523,0.568796,0.578500,0.479487,0.524360,0.614958
1,0.613833,0.666984,0.644633,0.731410,0.685285,0.721261
2,0.575922,0.702892,0.689509,0.728846,0.708632,0.768639
3,0.541324,0.732761,0.713861,0.769231,0.740512,0.800640
4,0.492106,0.768669,0.750602,0.798718,0.773913,0.840563


In [25]:
pd.DataFrame(history['val'])

,accuracy,precision,recall,f1,roc_auc
0,0.650964,0.602985,0.870690,0.712522,0.743122
1,0.691649,0.707547,0.646552,0.675676,0.778687
2,0.732334,0.698885,0.810345,0.750499,0.820084
3,0.723769,0.737327,0.689655,0.712695,0.815756
4,0.749465,0.693603,0.887931,0.778828,0.848826


In [26]:
evaluate(model, test_loader, device, num_classes)

{'accuracy': 0.7311725452812202,
 'precision': 0.6853582554517134,
 'recall': 0.8461538461538461,
 'f1': 0.757314974182444,
 'roc_auc': 0.8217682128835249}